# Step by Step: Wrangling and Tidying Data from SQLite Tables

This notebook demonstrates a systematic approach to wrangle and tidy data from SQLite tables using pandas.

## 1. Load Data from SQLite Tables

Use `pandas.read_sql_query` to load the tables (`students`, `courses`, `student_jobs`) from the SQLite database into DataFrames.

In [ ]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
con = sqlite3.connect("cademycode.db")

# Load tables into DataFrames
students = pd.read_sql_query("SELECT * FROM cademycode_students", con)
courses = pd.read_sql_query("SELECT * FROM cademycode_courses", con)
student_jobs = pd.read_sql_query("SELECT * FROM cademycode_student_jobs", con)

## 2. Inspect DataFrames

Use `.head()`, `.info()`, and `.describe()` to get an overview of the data and identify potential issues.

In [ ]:
# Inspect students DataFrame
print(students.head())
print(students.info())
print(students.describe(include='all'))

# Inspect courses DataFrame
print(courses.head())
print(courses.info())
print(courses.describe(include='all'))

# Inspect student_jobs DataFrame
print(student_jobs.head())
print(student_jobs.info())
print(student_jobs.describe(include='all'))

## 3. Check Data Types

Use `.dtypes` to review column data types and ensure they match expected formats.

In [ ]:
print("Students Data Types:")
print(students.dtypes)

print("\nCourses Data Types:")
print(courses.dtypes)

print("\nStudent Jobs Data Types:")
print(student_jobs.dtypes)

## 4. Remove Duplicate Rows

Use `.drop_duplicates(inplace=True)` to remove duplicate entries from each DataFrame.

In [ ]:
students.drop_duplicates(inplace=True)
courses.drop_duplicates(inplace=True)
student_jobs.drop_duplicates(inplace=True)

## 5. Handle Missing Values

Use `.isna().sum()` to identify missing values and decide on strategies (drop, fill, or impute) for handling them.

In [ ]:
print("Missing values in students:")
print(students.isna().sum())

print("\nMissing values in courses:")
print(courses.isna().sum())

print("\nMissing values in student_jobs:")
print(student_jobs.isna().sum())

# Example: Drop rows with missing values (customize as needed)
students.dropna(inplace=True)
courses.fillna("Unknown", inplace=True)
student_jobs.fillna(0, inplace=True)

## 6. Standardize Column Names

Rename columns to follow a consistent naming convention using `.rename()` or by assigning to `.columns`.

In [ ]:
# Example: Convert all column names to lowercase and replace spaces with underscores
students.columns = [col.lower().replace(" ", "_") for col in students.columns]
courses.columns = [col.lower().replace(" ", "_") for col in courses.columns]
student_jobs.columns = [col.lower().replace(" ", "_") for col in student_jobs.columns]

## 7. Convert Data Types if Necessary

Use `.astype()` to convert columns to appropriate data types (e.g., dates, categories, integers).

In [ ]:
# Example: Convert 'enrollment_date' in students to datetime
if 'enrollment_date' in students.columns:
    students['enrollment_date'] = pd.to_datetime(students['enrollment_date'], errors='coerce')

# Example: Convert 'course_id' in courses to category
if 'course_id' in courses.columns:
    courses['course_id'] = courses['course_id'].astype('category')

## 8. Reshape Data (if needed)

Use pandas functions like `.melt()`, `.pivot()`, or `.stack()` to reshape data for analysis.

In [ ]:
# Example: Pivot student_jobs to see job counts per student
if 'student_id' in student_jobs.columns and 'job_title' in student_jobs.columns:
    job_counts = student_jobs.pivot_table(index='student_id', columns='job_title', aggfunc='size', fill_value=0)
    print(job_counts.head())

## 9. Merge DataFrames

Use `pandas.merge()` to combine DataFrames based on common keys for integrated analysis.

In [ ]:
# Example: Merge students with student_jobs on 'student_id'
if 'student_id' in students.columns and 'student_id' in student_jobs.columns:
    students_jobs_merged = pd.merge(students, student_jobs, on='student_id', how='left')

# Example: Merge with courses if relevant key exists
if 'course_id' in students.columns and 'course_id' in courses.columns:
    full_data = pd.merge(students_jobs_merged, courses, on='course_id', how='left')

## 10. Save Cleaned Data

Export the cleaned and tidied DataFrames to CSV or other formats using `.to_csv()`.

In [ ]:
students.to_csv("cleaned_students.csv", index=False)
courses.to_csv("cleaned_courses.csv", index=False)
student_jobs.to_csv("cleaned_student_jobs.csv", index=False)

# Save merged DataFrame if created
if 'full_data' in locals():
    full_data.to_csv("cleaned_full_data.csv", index=False)